In [ ]:
# Import necessary packages
import os.path
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import datetime
import matplotlib.pyplot as plt
import warnings
import math
import glob
from scipy.interpolate import interp1d
warnings.filterwarnings("ignore")
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
# Pick a Wednesday 
node = '16'
path = # Removed
node_16 = pd.read_csv(path, names= ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])
node_16.head(5)



In [ ]:
def get_df_from_xml(path, legno, laneno, start_date, end_date):
    """
    Convert a xml file for Saturation Flow in SPaT into a Pandas DataFrame
    
    Parameters
    ----------
    path       : str (filepath to xml)
    legno      : int
    laneno     : int
    start_date : datetime object
    end_date   : datetime object
    """
    
    # Extract timestamp and time delay, sec
    tree = ET.parse(path)
    root = tree.getroot()
    time = [] # timestamp
    startupLT = [] # time delay in seconds
    for satflow in root.iter('SatFlow'):
        if ((satflow.attrib['legno']==str(legno)) & 
            (satflow.attrib['laneno']==str(laneno))):        
            time.append(float(satflow.attrib['time']))
            startupLT.append(float(satflow.attrib['startupLT'])) 

    # Convert the unix timestamp to human date
    sf_data = pd.DataFrame()
    sf_data['unixtime'] = time
    sf_data['satflow'] = startupLT
    sf_data['datetime'] = sf_data['unixtime'].apply(
        lambda x: datetime.datetime.fromtimestamp(x)) 

    # Filter out: 
    #      - data out of date range
    #      - data with negative saturation flow
    sf_data = sf_data[(sf_data['datetime'] >= start_date) 
                    & (sf_data['datetime'] <= end_date)
                    & (sf_data['satflow'] >= 0)].sort_values(
                        by='datetime').reset_index(drop=True)
    
    return sf_data 

In [ ]:
tree = ET.parse(path)
root = tree.getroot()

In [ ]:
path

In [ ]:
counts = [] # counts
time = [] # unix times
deltime = [] # unix times for delay
delay = [] #avg delay

for aggperf in root.iter('AggPerf'):
    if (aggperf.attrib['health']=='1'):
        for leg in aggperf.iter('Leg'):
            if(leg.attrib['no']=='4'):
                # Avg delay based on all lanes, but should be fine for verification purposes?
                for delaymeasures in leg.iter('DelayMeasures'):
                    deltime.append(aggperf.attrib['time'])
                    delay.append(delaymeasures.attrib['avgDelay'])
                for lanecounts in leg.iter('LaneCounts'):
                    if ((lanecounts.attrib['laneNo']!='1') & (lanecounts.attrib['laneNo']!='5') 
                        & (lanecounts.attrib['laneType'] == 'IB')):
                        counts.append(lanecounts.attrib['counts'])
                        time.append(aggperf.attrib['time'])
            


In [ ]:
# Create dataframe with time and vehicle count
data = pd.DataFrame()
data['unixtime'] = time
data['time'] = data['unixtime'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)))
data['count'] = counts
data.head()

In [ ]:
delays = pd.DataFrame()
delays['delay'] = delay
delays['delay'] = delays['delay'].apply(lambda x: float(x))
delays['unixtime'] = deltime
delays['time'] = delays['unixtime'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)))
delays['hour'] = delays['time'].apply(lambda x: x.hour)
delays = delays.replace([0, 1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23], 
                  [17, 18 ,19, 20, 21,22,23,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
delays.head()

In [ ]:
# Index at which the hour changes
idx_hr = delays['hour'].diff()[delays['hour'].diff() != 0].index.values
idx_hr = np.append(idx_hr,len(delays)-1)
idx_hr

In [ ]:
# Create plots of delay distributions at each hour, aggregated data
prev_idx = 0
for idx in idx_hr:
    hour = str(delays.iloc[idx]['hour'])
    plt.figure(figsize = (10,6))
    sns.distplot(delays.iloc[prev_idx:idx]['delay'], norm_hist= True)
    plt.xlabel("Delay (seconds)")
    plt.ylabel("Density")
    plt.title("Distribution of Delay at Hour " + hour + " (Ver)")
    filename = r'\Node' + node + 'Hour' + hour + '.png'
    plt.savefig(#Removed + filename)
    prev_idx = idx